<a href="https://colab.research.google.com/github/Song-yiJung/DH-Buddhist-Analysis-Tutorial/blob/main/6_%ED%95%9C%EB%AC%B8%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A1%9C_Word2Vec%EB%AA%A8%EB%8D%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📜 단어에 '의미'를 부여하는 Word2Vec**

TF-IDF는 문서 내에서 통계적으로 '중요한' 단어를 추출하는 지만, **단어의 '의미'를 모른다**는 결정적인 한계를 가진다.

TF-IDF에게 왕(王)과 임금은 그저 철자가 다른, 완전히 별개의 단어이다. 즉, 유학(儒學)과 성리학(性理學)의 관계를 전혀 알지 못한다.

**Word2Vec**은 이러한 한계를 뛰어넘기 위해, "단어의 의미 자체를 숫자로 표현할 수 없을까?"라는 고민에서 출발한 대표적인 **예측 기반 임베딩(Prediction-based Embedding)**이다.

### 1. **Word2Vec**의 핵심 개념: **"문맥이 의미를 만든다"**
Word2Vec의 가장 핵심적인 철학은 언어학자 J.R. Firth의 유명한 말, "단어의 의미는 그 단어 주변에 함께 등장하는 단어들을 보면 알 수 있다 (You shall know a word by the company it keeps)" 이다.

예) 어떤 사람 'A'가 항상 '궁궐', '왕비', '신하'라는 사람들과 어울린다면, 우리는 'A'가 '왕' 또는 그와 비슷한 신분일 것이라고 추측할 수 있다.

임금이라는 단어가 문헌 속에서 왕, 궁궐, 옥새, 신하 등의 단어와 자주 함께 등장한다면, 컴퓨터는 임금의 의미가 이 주변 단어들과 밀접한 관련이 있다고 학습한다.

Word2Vec은 이 아이디어를 수학적으로 구현한 것이다. 각 단어를 **수백 차원의 좌표(벡터)**를 가진 공간 속의 한 점으로 표현한다. 이 과정에서 비슷한 문맥을 가진 단어들은 '의미 공간'에서 서로 가까운 곳에 위치하게 된다.

### 2. 등장 배경: 왜 Word2Vec이 필요했나?

* 누가/언제: Word2Vec은 2013년, 구글의 토마스 미콜로프(Tomas Mikolov) 연구팀에 의해 개발되었다.

* 왜: 2010년대 초반, 딥러닝 기술이 부상하고 처리할 수 있는 데이터의 양이 폭발적으로 증가하면서, 기존의 통계 기반 모델(BoW, TF-IDF)의 한계가 명확해졌다. 더 나은 기계 번역, 감성 분석 등을 위해서는 컴퓨터가 단어의 '의미'와 '관계'를 이해하는 것이 필수적이었다. Word2Vec은 방대한 텍스트로부터 단어의 의미 벡터를 매우 효율적으로 학습할 수 있는 방법을 제시하며 자연어 처리 분야에 혁명을 가져왔다.

* 어떻게 의미를 학습하는가? (CBOW와 Skip-gram)
Word2Vec은 인공신경망을 이용해 두 가지 방식으로 '추측 게임'을 하며 단어를 학습한다.

 1) CBOW (Continuous Bag-of-Words): 주변 단어로 중심 단어 맞추기 (빈칸 채우기)

 인공신경망에게 "왕은 [ ? ]에게 명을 내렸다." 와 같은 문장을 주고, 주변 단어('왕은', '에게')를 통해 빈칸에 들어갈 단어 '신하'를 예측하도록 훈련시킨다.

 2) Skip-gram: 중심 단어로 주변 단어 맞추기 (이웃 추측하기)

 인공신경망에게 '신하'라는 단어를 주고, 그 주변에 등장할 확률이 높은 단어들('왕은', '에게')을 예측하도록 훈련시킨다.

이런 식의 예측 훈련을 수억, 수십억 번 반복하면서, 비슷한 예측 게임에 등장하는 단어들은 자연스럽게 서로 유사한 벡터 값을 갖게 된다.

## 📜 Word2Vec 실습: '추측 게임'으로 단어의 의미를 학습하는 과정

🎯 실습 목표

1. 아주 작은 문서 묶음(코퍼스)을 직접 만들어 본다.

2. 동일한 데이터로 **CBOW**와 **Skip-gram** 모델을 각각 훈련시킨다.

3. 훈련된 모델에게 "이 단어와 비슷한 의미의 단어는 뭐야?"라고 질문하여, 컴퓨터가 문맥을 통해 단어의 유사성을 정말 학습했는지 확인한다.

### ⚙️ 전체 실습 코드
아래 코드를 복사하여 Colab이나 로컬 환경에서 직접 실행한다.

In [1]:
# 1. 필요 라이브러리 불러오기

!pip install gensim
from gensim.models import Word2Vec

# 2. 실습용 데이터 준비 (작은 문서 묶음)
# 왕, 여왕, 신하, 백성 등의 단어가 비슷한 문맥에 등장하도록 구성
corpus = [
    '왕은 신하에게 명을 내렸다',
    '여왕은 백성에게 명을 내렸다',
    '왕은 백성을 사랑했다',
    '여왕은 신하를 신뢰했다'
]

# Word2Vec 모델은 '토큰화된 문장'들의 리스트를 입력으로 받습니다.
# 문장을 띄어쓰기 기준으로 잘라 리스트로 만들어 줍니다.
tokenized_corpus = [sentence.split() for sentence in corpus]
# 결과: [['왕은', '신하에게', ...], ['여왕은', '백성에게', ...], ...]

print("토큰화된 데이터:", tokenized_corpus)
print("-" * 50)


# 3. Word2Vec 모델 훈련시키기

# 3-1. CBOW 모델 훈련 (sg=0)
# '주변 단어로 중심 단어 맞추기' 방식으로 학습
model_cbow = Word2Vec(sentences=tokenized_corpus, sg=0, vector_size=10, window=2, min_count=1)

# 3-2. Skip-gram 모델 훈련 (sg=1)
# '중심 단어로 주변 단어 맞추기' 방식으로 학습
model_sg = Word2Vec(sentences=tokenized_corpus, sg=1, vector_size=10, window=2, min_count=1)


# 4. 결과 확인하기: 단어 유사도 검증

print("▼▽▼▽ 훈련된 모델 성능 확인 ▼▽▼▽\n")

# CBOW 모델에게 '왕은'과 가장 유사한 단어를 물어보기
similar_words_cbow = model_cbow.wv.most_similar('왕은')
print(f"CBOW 모델이 찾은 '왕은'과 유사한 단어: {similar_words_cbow}")

# Skip-gram 모델에게 '왕은'과 가장 유사한 단어를 물어보기
similar_words_sg = model_sg.wv.most_similar('왕은')
print(f"Skip-gram 모델이 찾은 '왕은'과 유사한 단어: {similar_words_sg}")

print("-" * 50)

# CBOW 모델에게 '신하에게'와 가장 유사한 단어를 물어보기
similar_words_cbow_subject = model_cbow.wv.most_similar('신하에게')
print(f"CBOW 모델이 찾은 '신하에게'와 유사한 단어: {similar_words_cbow_subject}")

# Skip-gram 모델에게 '신하에게'와 가장 유사한 단어를 물어보기
similar_words_sg_subject = model_sg.wv.most_similar('신하에게')
print(f"Skip-gram 모델이 찾은 '신하에게'와 유사한 단어: {similar_words_sg_subject}")

토큰화된 데이터: [['왕은', '신하에게', '명을', '내렸다'], ['여왕은', '백성에게', '명을', '내렸다'], ['왕은', '백성을', '사랑했다'], ['여왕은', '신하를', '신뢰했다']]
--------------------------------------------------
▼▽▼▽ 훈련된 모델 성능 확인 ▼▽▼▽

CBOW 모델이 찾은 '왕은'과 유사한 단어: [('백성을', 0.2941223680973053), ('백성에게', 0.20713211596012115), ('여왕은', 0.10494355112314224), ('신하를', 0.09267307072877884), ('명을', -0.1055101752281189), ('사랑했다', -0.11387499421834946), ('내렸다', -0.21133744716644287), ('신하에게', -0.2315719872713089), ('신뢰했다', -0.36627137660980225)]
Skip-gram 모델이 찾은 '왕은'과 유사한 단어: [('백성을', 0.2941223680973053), ('백성에게', 0.20713211596012115), ('여왕은', 0.10494355112314224), ('신하를', 0.09267307072877884), ('명을', -0.1055101752281189), ('사랑했다', -0.11387499421834946), ('내렸다', -0.21133744716644287), ('신하에게', -0.2315719872713089), ('신뢰했다', -0.36627137660980225)]
--------------------------------------------------
CBOW 모델이 찾은 '신하에게'와 유사한 단어: [('신하를', 0.2914133667945862), ('명을', 0.05541810393333435), ('신뢰했다', 0.042647670954465866), ('백성에게', -0.02176340483129024

### 📊 1. 결과에 대한 솔직한 분석: 왜 기대와 달랐을까?

먼저, 우리가 기대했던 이상적인 결과(예: '왕은'과 가장 비슷한 단어 1순위가 '여왕은')와 약간 다른 결과가 나온 점을 주목하는 것이 중요하다.

'왕은'과 가장 유사한 단어로 '백성을'이 '여왕은'보다 더 높은 점수를 받았다.

'신하에게'와 유사한 단어로 '백성에게'는 거의 관련이 없는 것(점수 -0.02)으로 나왔다.

그 이유는 **학습 데이터가 압도적으로 부족하기 때문**이다.

컴퓨터는 단 4개의 문장만을 보고 '왕', '여왕', '신하', '백성'의 의미를 학습해야 했다. 이는 마치 외국인이 한국어를 배우기 위해 책 4줄만 읽고 모든 단어의 뉘앙스를 파악하려는 것과 같다.

### 🧠 2. 개념 원리 다시 이해하기: 데이터가 부족할 때 생기는 일

Word2Vec은 텍스트에 나타난 통계적 패턴을 학습하여 단어의 의미를 '추측'한다. 우리의 작은 데이터셋에서 컴퓨터가 어떤 패턴을 발견했는지 살펴보겠다.

**학습 데이터**:

왕은 신하에게 명을 내렸다

여왕은 백성에게 명을 내렸다

왕은 백성을 사랑했다

여왕은 신하를 신뢰했다

* 왜 '왕은'의 1순위가 '백성을'일까?

 컴퓨터의 추론 과정은 다음과 같다.

1. '왕은'이라는 단어의 주변을 보니, 3번 문장 왕은 백성을 사랑했다에서 '백성을'이라는 단어와 아주 가까이 붙어 있다.

2. 반면, '여왕은'은 '왕은'과 같은 문장에 등장한 적이 단 한 번도 없습니다.

3. 단 4개의 문장만 본 컴퓨터에게는, **같은 문장에서 함께 등장했다는 사실('왕은'과 '백성을')**이, 다른 문장에서 비슷한 역할을 한다는 사실('왕은'과 '여왕은')보다 훨씬 더 강력한 통계적 증거가 된다.

4. 그래서 컴퓨터는 "아, '왕은'이라는 단어는 '백성을'이라는 단어와 매우 밀접한 관련이 있구나!"라고 결론 내리고 가장 높은 유사도 점수를 준 것이다.

5. 만약 수천, 수만 개의 문장을 학습시킨다면, '왕은'과 '여왕은'이 공통적으로 '명령하다', '다스리다', '궁궐에 살다' 등 훨씬 더 다양한 문맥을 공유하게 되면서, 우리가 기대하는 대로 '여왕은'의 유사도 점수가 가장 높아지게 될 것이다.



### 📝 3. 실제 결과 한 줄씩 뜯어보기

이제 실제 출력된 결과를 이 관점에서 자세히 해석해 보겠다.

**'왕은'과 유사한 단어 분석**:

1위 '백성을' (점수: 0.294): 위에서 설명했듯, 3번 문장에서 직접적으로 함께 등장했기 때문에 가장 높은 점수를 받았다.

3위 '여왕은' (점수: 0.104): 모델이 '왕은'과 '여왕은'이 문장의 주어 자리에 오고, 비슷한 구조(예: ... 명을 내렸다)를 가진다는 것을 어느 정도 학습했음을 보여준다. 하지만 데이터가 부족하여 그 신호가 약하게 잡힌 것이다.

음수(-) 점수들: 점수가 음수인 단어들은 이 작은 데이터 안에서 '왕은'과 거의 정반대의 문맥에 등장했음을 의미한다. 즉, '관련 없음' 또는 '의미 공간의 반대편'에 위치한 단어라고 컴퓨터가 판단한 것이다.

**'신하에게'와 유사한 단어 분석**:

1위 '신하를' (점수: 0.291): 컴퓨터가 '신하'라는 동일한 어근을 공유하고 비슷한 맥락에 쓰인다는 것을 매우 잘 학습했다. 이것은 성공적인 결과이다.

4위 '백성에게' (점수: -0.021): 점수가 0에 가깝고 심지어 음수이다. 왜일까? 우리의 데이터에서 '신하에게'는 '왕은'과 짝을 이루고, '백성에게'는 '여왕은'과 짝을 이룬다. 이 둘이 함께 등장하는 문장이 없었기 때문에, 컴퓨터는 이 두 단어의 의미적 유사성을 발견할 통계적 근거를 전혀 찾지 못한 것이다.

### ✨ 4. 결론: 디지털 인문학 연구자를 위한 교훈

이번 실습의 "기대에 못 미치는" 결과는 2가지 이해를 준다.

1. 데이터의 양과 질이 분석 결과를 결정한다: Word2Vec과 같은 예측 기반 모델의 성능은 전적으로 학습 데이터에 의존한다. 조선 시대 '왕'의 의미를 알려면, 사료 4줄이 아니라 조선왕조실록 전체를 보여줘야 하는 것과 같다. 적은 데이터로 얻은 결과는 '틀린' 것이 아니라, 그 데이터 안에서만 유효한 '편향된' 결과일 뿐이다.

2. 결과에 대한 비판적 사고의 중요성: 디지털 인문학 연구자는 컴퓨터가 뱉어낸 숫자를 맹신해서는 안 된다. 결과가 왜 이렇게 나왔는지 그 원인을 파악하고, 자신의 연구 분야 지식(Domain Knowledge)을 바탕으로 결과를 비판적으로 해석하는 능력이 필수적이다. 이번 경우, 우리는 "데이터가 부족해서"라고 명확히 원인을 진단할 수 있었다.

이 코드 실습은 Word2Vec이 어떻게 작동하는지를 보여줌과 동시에, 성공적인 디지털 인문학 연구가 컴퓨터의 계산 능력과 연구자의 깊이 있는 인문학적 통찰력의 결합을 통해 이루어짐을 명확히 보여주는 예시이다.